BandAid Beta Buddy - Options for No Stitching of Pre/Post
=====

An Automatic Beta Cell Analysis Pipeline 
---

Stitches Pre and Post & Only Segments Max Values of Pre and Post Images
---


<hr>

#### Written with equal contibution by: Anne Alsup *anne.alsup@mavs.uta.edu* & Kelli Fowlds *kelli.fowlds@mavs.uta.edu*.

Major compontents of the Cellpose script was written by Pradeep Rajasekhar. The [Cellpose GitHub](https://github.com/MouseLand/cellpose) along with the original [GoogleColab](https://colab.research.google.com/github/MouseLand/cellpose/blob/main/notebooks/Cellpose_cell_segmentation_2D_prediction_only.ipynb) by Pradeep are linked. 

Please read through all instructions given in each step cell. This a customizable pipeline, but the given format must be followed!

If you run into any issues, please let us know in our [BetaBuddy GitHub](https://github.com/jacobluber/BetaBuddy)!


## Step 1. nd2 to TIFF conversion
<hr>

1. The BBB_Cellpose directory will be emptied before each run. **Save or move any images you want to keep before running this step.**


2. If you already have TIFF file: 
    + Comment out (add a *#* before the line) the two lines that begin with *!./bftools/bfconvert* in **Cell1**.
    

3. If you already have TIFF files you will have to: 
    + Comment out (add a *#* before the line) the two lines that begin with *!./bftools/bfconvert* in **Cell1**.
    
    
    
4. **If you have a different file type, please use the *!./btftools/bfcovert* to convert your file to TIFF**


5. Change the arguements for bftools
    + There are arguements for Pre and Post images along with their respective DAPI stain.
    + Change the *./1V_Ca_Spike_Post.nd2* and/or *./DAPI_Post.nd2* to your ND2 file name(s). Keep the **./** before your file name as this lets the program know you file is located in the current directory.
    + Change the *1VSpiking.tif* and/or *1VDAPI.tif* name(s) to what you want your TIFF to be named. This arguement does *not* need the **./**.


*Note: Your filenames cannot have any spaces when using a LINUX OS. Replacing spaces with '_' is common practice.* <br>


In [ ]:
# Cell1
############################################################################################
################################### Follow this format: ####################################
############# !./bftools/bfconvert -overwrite ./FILENAME.nd2 NEWFILENAME.tif ###############
############################################################################################

!./bftools/bfconvert -overwrite ./DAPI_Pre.nd2 1VDAPIPre.tif 
!./bftools/bfconvert -overwrite ./1V_Ca_Spike_Pre.nd2 1VPre.tif
# !./bftools/bfconvert -overwrite ./DAPI_Post.nd2 1VDAPIPost.tif 
# !./bftools/bfconvert -overwrite ./1V_Ca_Spike_Post.nd2 1VPost.tif

import os
import shutil

# Emptying the CellPoseImg directory that will be the output directory for the merged image
imgsave_dir = "./BBB_Cellpose"
if not os.path.exists(imgsave_dir):
    print("No BBB_Cellpose directory found. Creating a new one.")
    os.mkdir(imgsave_dir)
else:
  print("Existing BBB_Cellpose directory found. Deleting it.")
  shutil.rmtree(imgsave_dir)
  os.mkdir(imgsave_dir)

##  Step 2. Merge single DAPI stain with max fluorescent image </h2>
<hr>



* The input argument for this step will follow this format **'/my/path/end/with/slash/ 1VDAPIPre 1VDAPIPost'**
    + Every arguement is seperated with a space. Follow the steps below for all 3 arguements.
    + *Do **NOT** add the image format (.tif) to the end of the file name*

* Comment out the following if you don't have two image sets:
    + *impost = imread(dir_path+'/1VPost.tif')* 
    + *B = np.max(impost,axis=0)* 
    + *cv2.imwrite(dir_path+"/BBB_Post.png", B)* 

1. Your current directory path **ENDING** with a **BACKSLASH** (/my/path/end/with/slash/).
2. Arguement for if you are stitching Control and Experimental Images.
    + Add ***NoStitch*** if you are **NOT** including two different image sets. Still include a fourth arguement.
    + Add ***Stitch*** if you have two image sets (Pre/Post).
3. Your *Pre/Control* DAPI image stack name (1VDAPIPre).
4. Your *Post/Experimental* DAPI image name (1VDAPIPost).
    + *Note the arguement requires **4** inputs*

Your Cellpose images will be saved under **./BBB_Cellpose**. The control and experimental images will be saved as "BBB_*the name of your DAPI image*.tif".


In [ ]:
# Cell2
from skimage.io import imread
import cv2
import numpy as np

#get path for image loading
dir_path = os.getcwd()
print(dir_path)

#load stacked original images and mask images
###########################################
##### CHANGE THE TIF FILE NAMES BELOW #####
###########################################

imp = imread(dir_path+'/1VPre.tif') #Pre/Control image name! Include the backslash
# impost = imread(dir_path+'/1VPost.tif') #Post/Experimental image name! Include the backslash

#max and mean pixel value across all frames
A = np.max(imp,axis=0)
# B = np.max(impost,axis=0)
cv2.imwrite(dir_path+"/BBB_Pre.png", A)
# cv2.imwrite(dir_path+"/BBB_Post.png", B)


print('The max image for Pre and Post have been saved.')

###########################################
######### CHANGE THE INPUTS BELOW #########
###########################################

!./Fiji.app/ImageJ-linux64 --headless -macro ./BBB_DAPIMerge.ijm '/home/BetaBuddy/ NoStitch 1VDAPIPre PlaceHolder'

Step 3. Automatic cell segmentation with Cellpose 
----
<hr>

In [ ]:
# Cell3
# Configuring Cellpose, adding dependencies, and checking GPU access
# https://colab.research.google.com/github/MouseLand/cellpose/blob/main/notebooks/Cellpose_cell_segmentation_2D_prediction_only.ipynb
import numpy as np
import time, os, sys, random
from urllib.parse import urlparse
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
import ipywidgets as widgets
import numpy as np
from IPython.display import display
import shutil
import re

from cellpose import core, utils, models

if core.use_gpu()==False: 
  print("You do not have GPU access. Cellpose will run through CPU.")
  use_GPU=False
else:
  print("You have access to the GPU.")
  use_GPU=True

!nvidia-smi
print("All dependencies loaded")

### Step 3.1: Set parameters for image format, file type, and input directory 

<hr>

#### Widgets will appear in the output after running the cell. It's helpful to run the widget cells using *ctrl+Enter*, so Jupyter doesn't skip to the next cell.

#### Note: The button will become dark gray when selected.
1. Confirm the input directory is correct. 

2. Choose a model option 
    + TN2 and TN3 seem to work best with Beta-cell segmenting. 
    + Nuclei will work well for images with only DAPI staining. 
    + *Note: Only nuclei segmentation may not perform well for subsequent tracking and analysis steps.*

3. Choose the channel that contains the entire cell to segment. You can reference the images above to find the correct channel.
    + *Keep the channel at 0 if you only have one channel.*

4. Choose the channel with your DAPI stain. **If you do not have a DAPI stain, keep the channel at 0.**
5. Set Cell Probability Threshold  
    + Decrease this threshold if your masks are too small or if Cellpose is not detecting enough cells.
    + Increase this threshold if Cellpose is detecting too many cells. 

6. Set Flow Threshold 
    + Decrease this parameter if Cellpose is not detecting enough cells. 
    + Increase this parameter if Cellpose is detecting too many cells.      
 
7. Set Diameter
     + If you don't konw diameter, leave at 0. 
     + 70 pixels for 2048x2048 in Beta-cell images at 20X objective. 
     + Diameter will be in pixels.

In [ ]:
# Cell4
# Creating input widgets

###########################################
### RUN THIS CELL TO SELECT THE WIDGETS ###
###########################################

input_dir = os.getcwd() +'/BBB_Cellpose/'
print("Input directory: " + input_dir)

Model_Choice = widgets.ToggleButtons(
    options=['Cytoplasm', 'Cytoplasm2', 'Nuclei', 'TN2','TN3'],
    value = 'TN2',
    description='Select One', 
)
display(Model_Choice)

GPU = widgets.ToggleButtons(
    options=['CPU','GPU'],
    value = 'GPU',
    description = 'Select One')
display(GPU)

segchan=widgets.Dropdown(
    options=['0','1', '2', '3'],
    value='1',
    description='Cell Channel:',
    disabled=False,
)
display(segchan)


qDapi=widgets.Dropdown(
    options=['0','1', '2', '3'],
    value='2',
    description='DAPI Channel:',
    disabled=False,
)
display(qDapi)


cellprob=widgets.FloatSlider(
    value=0,
    min=-6, 
    max=6, 
    step=1.0, 
    description='Cell Probability'
)
display(cellprob)


flowthresh = widgets.FloatSlider(
    value=0.4,
    min=0.1,
    max=1.1, 
    step=0.1, 
    description='Flow Threshold'
)
display(flowthresh)


diam = widgets.IntText(
    value = 70,
    min = 0,
    max = 200,
    step = 1,
    description = "Diameter")
display(diam)



In [ ]:
# Cell5
fname1 = os.listdir(input_dir)
fname = [item for item in fname1 if item.endswith("tif")]    

imgsave_dir = input_dir


#######################################################
############ Creating/Emptying Mask folder ############
#######################################################

save_dir = input_dir+"Masks/"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
else:
    print("Existing Mask Directory found. Deleting it.")
    shutil.rmtree(save_dir)

    
if(len(fname)==0):
    print("Number of images loaded: %d." %(len(fname)))
    print("Cannot read image files. Check if folder has images")
else:
    print("Number of images loaded: %d." %(len(fname)))
    
#######################################################    
### Reading all images and displaying one at random ###
#######################################################

imgs = []
for im in range(len(fname)):
    im = skimage.io.imread(imgsave_dir + fname[im])
    n_dim=len(im.shape) #shape of image
    dim=im.shape #dimensions of image
    channel=min(dim) #channel will be dimension with min value usually
    channel_position=dim.index(channel)
    # If no of dim is 3 and channel is first index, swap channel to last index
    if n_dim==3 and channel_position==0: 
        im=im.transpose(1,2,0)
        dim=im.shape
    imgs.append(im)
print("Number of images read: " + str(len(imgs)))
print("Example Image:")
random_idx = random.choice(range(len(imgs)))
x=imgs[random_idx]
n_dim=len(x.shape)
file_name=os.path.basename(fname[random_idx])
print(file_name+" has "+str(n_dim)+" dimensions/s")
if n_dim==3:
    channel_image=x.shape[2]
    fig, axs = plt.subplots(1, channel_image,figsize=(12,5))
    print("Image: %s" %(file_name))
    for channel in range(channel_image):
        axs[channel].imshow(x[:,:,channel])
        axs[channel].set_title('Channel '+str(channel+1),size=5)
        axs[channel].axis('off')
    fig.tight_layout()
elif n_dim==2:
    print("One Channel")
    plt.imshow(x)
else:
    print("Channel number invalid or dimensions wrong. Image shape is: "+str(x.shape))

In [ ]:
# Cell7
import torch
print("Model choice is: " + Model_Choice.value)
print("Segmentation channel is: " + str(segchan.value))
if qDapi.value == 0:
    Use_nuclear_channel = False
    print("No DAPI Channel")
else:
    Use_nuclear_channel = True
    print("DAPI channel is: " + qDapi.value)
    
if GPU == 'GPU':
    gpuYN = True
else:
    gpuYN = False
        

print("Cell probability threshold: " + str(cellprob.value))
print("Flow threshold: " + str(flowthresh.value))
      
model_choice = Model_Choice.value
segment_channel = int(segchan.value)
nuclear_channel = int(qDapi.value)

#diameter = int(diam)

if model_choice=="Cytoplasm":
  model_type="cyto"
elif model_choice=="Cytoplasm2":
  model_type="cyto2"
elif model_choice=="Nuclei":
  model_type="nuclei" 
elif model_choice=="TN2":
    model_type="TN2"
elif model_choice=="TN3":
    model_type="TN3"
# channels = [cytoplasm, nucleus]

########################
#### ANNIE FIX THIS ####
########################
if model_choice not in "Nucleus":
  if Use_nuclear_channel:
    channels=[segment_channel,nuclear_channel]
  else:
    channels=[segment_channel,nuclear_channel]
else: #nucleus
  channels=[segment_channel,0]

if diam == 0:
    diam = None
    

# DEFINE CELLPOSE MODEL
model = models.CellposeModel(gpu=gpuYN, model_type=model_type)
print(model)
diameter = None
from skimage.util import img_as_ubyte

flow_threshold=flowthresh.value
cellprob_threshold=cellprob.value

# You can change the test image here
img1=imgs[0]
import cv2
# masks, flows, styles, diams = model.eval(img1,diameter=diameter,flow_threshold=flow_threshold,cellprob_threshold=cellprob_threshold, channels=channels)
# masks = model.eval(img1, channels=[segment_channel, nuclear_channel],
#                    diameter=diameter)[0]
# masks, flows, styles, diams = model.eval(img1, diameter=diameter, flow_threshold=flow_threshold,cellprob_threshold=cellprob_threshold, channels=channels)
masks, flows, styles = model.eval(img1, 
                                  channels=[segment_channel, nuclear_channel],
                                  diameter=diameter,
                                  flow_threshold=flow_threshold,
                                  cellprob_threshold=cellprob_threshold
                                  )
# DISPLAY RESULTS
from cellpose import plot
maski = masks
flowi = flows[0]

#convert to 8-bit if not so it can display properly in the graph
if img1.dtype!='uint8':
  img1=img_as_ubyte(img1)

fig = plt.figure(figsize=(24,8))
plot.show_segmentation(fig, img1, maski, flowi, channels=channels)
plt.tight_layout()
plt.show()

In [ ]:
# Cell8
# Save mask images in Masks directory under CellPoseImgs
print("Save Directory is: ",save_dir)
if (not os.path.exists(save_dir)):
    os.mkdir(save_dir)

for img_idx in range(len(fname)):
    file_name=fname[img_idx]
    print("\nSegmenting: ",file_name)
#     mask, flow, style, diam = model.eval(imgs[img_idx], diameter=diameter, flow_threshold=flow_threshold,cellprob_threshold=cellprob_threshold, channels=channels)
    masks, flows, styles = model.eval(imgs[img_idx], 
                                  channels=[segment_channel, nuclear_channel],
                                  diameter=diameter,
                                  flow_threshold=flow_threshold,
                                  cellprob_threshold=cellprob_threshold
                                  )
    # Save name for masks
    if file_name.endswith('.tif'):
        file_name = re.sub(".tif","",file_name)
    mask_output_name=save_dir+"MASK_"+file_name+".tif"
    # Save mask as 16-bit in case this has to be used for detecting than 255 objects
    # A 16-bit image will look black until opened in software such as ImageJ
    # Change "uint16" to "uint8" if you want the preview to be visible 
    mask=masks.astype(np.uint16)
    skimage.io.imsave(mask_output_name, mask, check_contrast=False)
 
print("\nSegmentation complete and files saved :)")

Step 4. Cell registration and tracking 
----
<hr>

1. The first terminal command will merge the mask images with their corresponding orginal beta cell image to begin cell registration. 
    + **You will need to change the *'/home/BetaBuddy/'* to your current directory**
    + 6 Arguments 
    
    1. Directory 
    2. ***NoStitch*** if you have one image set OR ***Stitch*** if you have two image sets.
    3. Target stain image name for control or main image (no .tif for any image names)
    4. DAPI stain for control or main image
    5. Target stain image name for experimental or any name if you have only one image set.
    6. DAPI stain image name for experimental or any name if you have one image set.
   
    
    
2. The second terminal command will push our mask/original image stack through the Fiji plugin TrackMate. A few short printouts should show up with a very long log of all the track locations and frames afterwards. 
</br> 



### Outputs

* An XML titled *MaskMerged.xml* will be saved in the current directory along with the *MaskMerged.tif*. You can load this into Fiji through *Plugins -> Tracking -> Load TrackMate File* and see the calculated tracks. You can also open the *Track* tab and click on TrackIDs to see their location within the TrackMate GUI. 
* Raw data will be saved in a CSV named "experimentjup.csv". 

*Note: The TrackIDs in the CSV files will be off by one.*


In [ ]:
# Cell9

###########################################
######### CHANGE THE INPUTS BELOW #########
###########################################
!./Fiji.app/ImageJ-linux64 --headless -macro ./BBB_MaskMerge.ijm '/home/BetaBuddy/ NoStitch 1VPre 1VDAPIPre 1VPost 1VDAPIPost'
!./Fiji.app/ImageJ-linux64 --headless ./BBB_TrackMate.py

Step 5. Background Subtraction 
----

<hr>

* Random pixel locations will be chosen and compared to every mask frame to ensure the location is not within a cell. 
* If the location is not within a cell, the pixel value from every frame will be saved in a csv named "backgroundsub". 
* This will be used in the next step for background subtraction for signal analysis.
* Go to the bottom & change file name inputs

In [ ]:
# Cell10

import os
from skimage import io
from matplotlib import pyplot as plt
import numpy as np
import csv
import random
from statistics import mean

#get path for image loading
dir_path = os.getcwd()
print(dir_path)

def backgroundsub(img,mask,NewCSV,path,spotnum = 100):
    #load stacked original images and mask images
    imspike = io.imread(dir_path+img) ##CHANGE NAME
    immask = io.imread(dir_path+mask)

    # Get dimensions for each image (frames, 900, 900, # of channels)
    dims=imspike.shape
    dimsm=immask.shape
    print("Image dimensions: " + str(dims))

    # Create array for pixel values of all 100 spots
    x = []
    for index in range(0,dims[0]):
        x.append([None]*spotnum)

    count = 0
    # Array to house locations that are not masks
    goodlocs=[]
    tmpx = []
    while count<spotnum:
        loc = []
        imgval = []
        #random pixel location
        x1tmp = random.randrange(0,int(len(imspike[1])))
        y1tmp = random.randrange(0,int(len(imspike[1])))
        imgloc = [int(x1tmp),int(y1tmp)]
        tmpx = np.array([None]*int(dims[0]))
        for img in range(dims[0]):
            #pixel intensity from all three channels
            imgval = imspike[img,imgloc[0],imgloc[1]] #8bit
            maimgval = immask[img,imgloc[0],imgloc[1]]
            # If intensity on mask image is 0, the pixel location is *not* within a cell 
            if np.all(maimgval==0):
                tmpx[img]=imgval 
            # If intensity on mask image is *not* 0, all temporary variables are emptied   
            else:
                maimagval = []
                imgval = []
                imgloc=[]
                break
        if None in tmpx:
          # Empty out temporary array if there are any "None"s left
          tmpx = []
        else: 
          # Save temporary array into large array will all intensity values  
            for do in range(dims[0]):
                x[do][count] = tmpx[do]
            # Save pixel location  
            goodlocs.append(imgloc)
            count += 1

    # Header row names for csv
    heads = []
    for namez in range(spotnum):
        heads.append('Spot '+ str(namez))

    # Saving pixel locations for reference
    spotz = []
    for locz in range(spotnum):
        spotz.append('('+str(goodlocs[locz][1])+'/'+str(goodlocs[locz][0])+')')

    print("Locations of background subtraction: " + str(spotz))

    # Save intensity values of all 100 pixel locations across all frames
    filename = dir_path+"/BBB_backgroundsub.csv"
    if NewCSV is True:
        with open(filename,'w') as csvfile:
            csvwriter = csv.writer(csvfile, delimiter = ',',lineterminator='\r')
            csvwrite2 = csv.writer(csvfile, delimiter = ' ',lineterminator='\r')
            csvwriter.writerow(heads)
            csvwriter.writerow(spotz)
            for i in range(len(x)):
                csvwriter.writerow(x[i])
    else:
        with open(filename,'a') as csvfile: ##Will append to previous csv from pre
            csvwriter = csv.writer(csvfile, delimiter = ',',lineterminator='\r')
            csvwrite2 = csv.writer(csvfile, delimiter = ' ',lineterminator='\r')
            for i in range(len(x)):
                csvwriter.writerow(x[i])
            
            
            
backgroundsub("/1VPre.tif","/BBB_1VPre_Mask.tif",True,dir_path,100) 
#"BBB_ + Pre_Name + _Mask.tif"
#"BBB_ + Post_Name + _Mask.tif"
#backgroundsub('PreImage.tif','/BBB_PreMask.tif',#ofspots,True -> new csv, False -> append to previous csv)

#############################################################
########## Uncomment below if using two image sets ##########
#############################################################

# backgroundsub("/1VPost.tif","/BBB_1VPost_Mask.tif", False,dir_path,100)   

## Step 6. Statistical Analysis 
<hr>

* We can now run the statistical analysis in an R script! 
* The cell below should create three figures and a CSV file, named Experimental_Data.csv, you can use in custom analyses. All will be saved in your current directory. 
* The final CSV file has only tracks that span the full image stack and identifies tracks in the top 20% of signal variance.
</ul>


In [ ]:
# Cell11

!Rscript Beta_Buddy_Stats_Analysis.R